In [7]:
%reset
import matplotlib.pyplot as plt
import rasterio 
from rasterio.plot import plotting_extent, show
from rasterio.merge import merge
import geopandas as gpd
from osgeo import gdal
import numpy as np
from skimage import io


def data_fusion_mspec(ortho_rgb, ortho_mspec, outputpath):
    
    ortho_rgb = ortho_rgb
    ortho_mspec = ortho_mspec
    
    c = gdal.Open(ortho_mspec)
    x = gdal.Open(ortho_rgb)
    d = gdal.Open(dem)
    
    dsReprj_mspec= gdal.Warp("D:/cbr/VP2/VP2_SoilMapping/CodeOutput/demRepr_mspec_5.tif", c, xRes = x.GetGeoTransform()[1], yRes= x.GetGeoTransform()[5])
    dsReprj_dem = gdal.Warp("D:/cbr/VP2/VP2_SoilMapping/CodeOutput/demRepr_dem_5.tif", d, xRes = x.GetGeoTransform()[1], yRes= x.GetGeoTransform()[5])
    dsReprj_rgb = gdal.Warp("D:/cbr/VP2/VP2_SoilMapping/CodeOutput/demRepr_rgb_5.tif", x, xRes = x.GetGeoTransform()[1], yRes= x.GetGeoTransform()[5])
    
    stack = []
    
    rgb_red = dsReprj_rgb.GetRasterBand(1).ReadAsArray().astype(np.float32)
    rgb_green = dsReprj_rgb.GetRasterBand(2).ReadAsArray().astype(np.float32)
    rgb_blue =  dsReprj_rgb.GetRasterBand(3).ReadAsArray().astype(np.float32)
    stack.append(rgb_red)
    stack.append(rgb_green)
    stack.append(rgb_blue)
    

    mspec_rededge = dsReprj_mspec.GetRasterBand(3).ReadAsArray().astype(np.float32) 
    mspec_nir = dsReprj_mspec.GetRasterBand(4).ReadAsArray().astype(np.float32) 
    mspec_red = dsReprj_mspec.GetRasterBand(2).ReadAsArray().astype(np.float32)
    mspec_ndvi = (mspec_nir - mspec_red)/(mspec_nir + mspec_red)
    stack.append(mspec_rededge)
    stack.append(mspec_nir)
    stack.append(mspec_ndvi)

    dem_chanel = dsReprj_dem.GetRasterBand(1).ReadAsArray().astype(np.float32)
    stack.append(dem_chanel)
    
    print("Channel count", len(stack))
    #stack_data = np.asarray(stack, dtype=object)
    stack_data = np.array(stack, dtype=object)

    yvalue = 0
    xvalue = 0
    for i in stack_data:
        if yvalue < i.shape[0]:
            yvalue = i.shape[0]

        if xvalue < i.shape[1]:
            xvalue = i.shape[1]
    
    print("Maximum Value: ", xvalue, "Minimum Value: ", yvalue)
   
   
    driver = gdal.GetDriverByName ( "GTiff" )
    DataSet = driver.Create(outputpath, xvalue, yvalue, len(stack_data), gdal.GDT_Float32)
    DataSet.SetGeoTransform(x.GetGeoTransform())
    DataSet.SetProjection(x.GetProjection())
    for i, image in enumerate(stack_data, 1):
        DataSet.GetRasterBand(i).WriteArray(image)
        DataSet.GetRasterBand(i).SetNoDataValue(-9999.0)
    DataSet.FlushCache()

    print("Finished     .tiff file created")
    return None

In [8]:
ortho_rgb   = "D:/cbr/VP2/VP2_SoilMapping/Processed/Wetzwil_Ortho/prediction2/Wetzwil_pred_rgb3.tif"
ortho_mspec = "D:/cbr/VP2/VP2_SoilMapping/Processed/Wetzwil_Ortho/prediction2/Wetzwil_pred_mspec3.tif"
dem         = "D:/cbr/VP2/VP2_SoilMapping/Processed/Wetzwil_Ortho/prediction2/Wetzwil_pred_dem3.tif"
outputpath2 = "D:/cbr/VP2/VP2_SoilMapping/Processed/Wetzwil_Ortho/prediction2/Stack.tif"

data_fusion_mspec(ortho_rgb, ortho_mspec, outputpath2)

C:\Users\ST1154~1\AppData\Local\Temp/ipykernel_10260/3925752296.py:41: RuntimeWarning: invalid value encountered in true_divide
  mspec_ndvi = (mspec_nir - mspec_red)/(mspec_nir + mspec_red)


Channel count 7
Maximum Value:  8125 Minimum Value:  6813
Finished     .tiff file created
